In [1]:
import autoencoder_conv2d

import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from tensorflow.keras.datasets import mnist
from clusteringlayer import ClusteringLayer
from tensorflow.keras.initializers import VarianceScaling
from sklearn.utils.linear_assignment_ import linear_assignment

/home/tanmay/anaconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape(x.shape + (1,))
x = np.divide(x, 255.)

In [4]:
autoencoder, encoder = autoencoder_conv2d.autoencoderConv2D_1()

In [5]:
autoencoder.summary()

Model: "AE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 14, 14, 32)        832       
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 64)          51264     
_________________________________________________________________
conv3 (Conv2D)               (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
embedding (Dense)            (None, 10)                11530     
_________________________________________________________________
dense (Dense)                (None, 1152)              12672    

In [6]:
pretrain_epochs = 20
batch_size = 256

In [7]:
autoencoder.compile(optimizer = 'adadelta', loss = 'mse')

In [8]:
autoencoder.fit(x, x, batch_size = batch_size, epochs = pretrain_epochs, verbose = 2)

Epoch 1/20
274/274 - 8s - loss: 0.1121
Epoch 2/20
274/274 - 2s - loss: 0.1118
Epoch 3/20
274/274 - 2s - loss: 0.1115
Epoch 4/20
274/274 - 2s - loss: 0.1110
Epoch 5/20
274/274 - 2s - loss: 0.1106
Epoch 6/20
274/274 - 2s - loss: 0.1101
Epoch 7/20
274/274 - 2s - loss: 0.1096
Epoch 8/20
274/274 - 2s - loss: 0.1091
Epoch 9/20
274/274 - 2s - loss: 0.1085
Epoch 10/20
274/274 - 2s - loss: 0.1079
Epoch 11/20
274/274 - 2s - loss: 0.1074
Epoch 12/20
274/274 - 2s - loss: 0.1068
Epoch 13/20
274/274 - 2s - loss: 0.1061
Epoch 14/20
274/274 - 2s - loss: 0.1055
Epoch 15/20
274/274 - 2s - loss: 0.1048
Epoch 16/20
274/274 - 2s - loss: 0.1041
Epoch 17/20
274/274 - 2s - loss: 0.1035
Epoch 18/20
274/274 - 2s - loss: 0.1028
Epoch 19/20
274/274 - 2s - loss: 0.1021
Epoch 20/20
274/274 - 2s - loss: 0.1014


In [9]:
# Add clustering layer

clustering_layer = ClusteringLayer(10, name = 'clustering')(encoder.output)
model = tf.keras.models.Model(inputs = encoder.input, outputs = clustering_layer)
model.compile(optimizer = 'adam', loss = 'kld')

In [10]:
# initialize cluster centers using k-means

kmeans = KMeans(n_clusters = 10, n_init = 20)
y_pred = kmeans.fit_predict(encoder.predict(x))

In [11]:
y_pred_last = np.copy(y_pred)

In [12]:
model.get_layer(name = 'clustering').set_weights([kmeans.cluster_centers_])